# 0. Install Dependencies

# IMPORTANT Make sure you create a conda environment with Python 3.7 installed 

In [1]:
!pip install tensorflow==2.3.0
!pip install gym==0.18.0
!pip install keras
!pip install keras-rl2
!pip install rl-agents==0.1.1
!pip install pyglet==1.5.11

  Using cached pyglet-1.5.0-py2.py3-none-any.whl (1.0 MB)
  Attempting uninstall: pyglet
    Found existing installation: pyglet 1.5.11
    Uninstalling pyglet-1.5.11:
      Successfully uninstalled pyglet-1.5.11
  Using cached pyglet-1.5.11-py3-none-any.whl (1.1 MB)
  Attempting uninstall: pyglet
    Found existing installation: pyglet 1.5.0
    Uninstalling pyglet-1.5.0:
      Successfully uninstalled pyglet-1.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gym 0.18.0 requires pyglet<=1.5.0,>=1.4.0, but you have pyglet 1.5.11 which is incompatible.


## IMPORTANT https://stackoverflow.com/questions/73978651/valueerror-error-when-checking-input-expected-flatten-input-to-have-shape-1 

# 1. Test Random Environment with OpenAI Gym

In [2]:
import gym 

In [3]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape
actions = env.action_space.n

observation_space = env.observation_space
action_space = env.action_space

print("states", states)
print("actions", actions)
print("observation space", observation_space)
print("action space", action_space)

states (4,)
actions 2
observation space Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)
action space Discrete(2)


In [4]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:45.0
Episode:2 Score:18.0
Episode:3 Score:26.0
Episode:4 Score:11.0
Episode:5 Score:11.0
Episode:6 Score:15.0
Episode:7 Score:9.0
Episode:8 Score:38.0
Episode:9 Score:10.0
Episode:10 Score:11.0


In [5]:
states[0],1

(4, 1)

# 2. Create a Deep Learning Model with Keras

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten,Input
from tensorflow.keras.optimizers import Adam


In [7]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states[0])))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [16]:
# sometimes, when we get the following error, we need to delete the model and build it 
# 
# AttributeError: 'Sequential' object has no attribute '_compile_time_distribution_strategy'


if "model" in locals():
    del model

In [17]:
model = build_model(states, actions)

# 3. Build Agent with Keras-RL

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 24)                120       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [11]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [12]:
# Example of an observation from the environment
observation = env.reset()  # This should be a 1D array with shape (4,)
observation.shape

(4,)

In [13]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [18]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=10000, visualize=False, verbose=1)

Training for 10000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 12:42 - reward: 1.0000

/Users/siddharthmanu/anaconda3/envs/keras-rl/lib/python3.7/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 64s 6ms/step - reward: 1.0000
done, took 63.946 seconds


In [19]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 226.000, steps: 226
Episode 2: reward: 174.000, steps: 174
Episode 3: reward: 187.000, steps: 187
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 178.000, steps: 178
Episode 6: reward: 191.000, steps: 191
Episode 7: reward: 237.000, steps: 237
Episode 8: reward: 329.000, steps: 329
Episode 9: reward: 207.000, steps: 207
Episode 10: reward: 304.000, steps: 304
Episode 11: reward: 289.000, steps: 289
Episode 12: reward: 190.000, steps: 190
Episode 13: reward: 174.000, steps: 174
Episode 14: reward: 185.000, steps: 185
Episode 15: reward: 175.000, steps: 175
Episode 16: reward: 194.000, steps: 194
Episode 17: reward: 199.000, steps: 199
Episode 18: reward: 377.000, steps: 377
Episode 19: reward: 258.000, steps: 258
Episode 20: reward: 271.000, steps: 271
Episode 21: reward: 168.000, steps: 168
Episode 22: reward: 212.000, steps: 212
Episode 23: reward: 204.000, steps: 204
Episode 24: reward: 180.000, steps: 180
Episode 25: reward: 

In [21]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...


2024-05-27 00:49:12.889 python[40576:4040988] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fdea6bfc910>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2024-05-27 00:49:12.890 python[40576:4040988] Warning: Expected min height of view: (<NSButton: 0x7fdea69b77e0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2024-05-27 00:49:12.892 python[40576:4040988] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fdea69bb480>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2024-05-27 00:49:12.894 python[40576:4040988] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fdea69c2e10>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.


Episode 1: reward: 317.000, steps: 317
Episode 2: reward: 205.000, steps: 205
Episode 3: reward: 189.000, steps: 189
Episode 4: reward: 196.000, steps: 196
Episode 5: reward: 212.000, steps: 212
Episode 6: reward: 328.000, steps: 328
Episode 7: reward: 233.000, steps: 233
Episode 8: reward: 173.000, steps: 173
Episode 9: reward: 177.000, steps: 177
Episode 10: reward: 199.000, steps: 199
Episode 11: reward: 291.000, steps: 291
Episode 12: reward: 207.000, steps: 207
Episode 13: reward: 171.000, steps: 171
Episode 14: reward: 215.000, steps: 215
Episode 15: reward: 185.000, steps: 185


# 4. Reloading Agent from Memory

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=False)

In [ ]:
!pip show pyglet